# Web Scraping Intro

Sometimes, data viewable on the web may not be available through and API or downloadable as csv or otherwise. In these cases, scraping that data might be possible.

In this notebook, we'll see how we can retrieve the contents of a website and then parse the resulting HTML to extract the data we want.

For this, we'll again be using the [_requests_](https://requests.readthedocs.io/en/master/) library.

In [ ]:
import requests

Let's say we want to pull some data from http://en.wikipedia.org/wiki/Turing_Award. 

I can start by sending a get request for the contents of the site.

In [ ]:
URL = 'http://en.wikipedia.org/wiki/Turing_Award'

response = requests.get(URL)

We can check the status code using the `status_code` attribute.

In [ ]:
response.status_code

A 200 status code is the standard response for a successful request.  

Let's see what this request returned.

In [ ]:
response.text

It is very hard to decipher the above text. Luckily for us, the [_Beautiful Soup_](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library comes to the rescue. This library assists us in parsing HTML into something usable.

In [ ]:
from bs4 import BeautifulSoup

First, we can soupify our response text. Since we are working with HTML, we can specify that we need the html parser.

In [ ]:
soup = BeautifulSoup(response.text)

Now, we can print it out in a slightly more readable form.

In [ ]:
print(soup.prettify())

What we are looking at is the HTML for this page. This is rendered by your browser into the Wikipedia page that you see.

<img src="../assets/html.png">


If you navigate to this page in your browser, you can view page source or inspect elements to see the underlying HTML.

Beautiful Soup lets us search through this HTML and extract out the contents we want by tag.  

Say we wanted to find the title of this page. We can accomplish this by using the `.find` method on our soup, telling it that we want to find the first `title` tag.

In [ ]:
soup.find('title')

Notice that this returns a bs4 Tag object.

In [ ]:
type(soup.find('title'))

To extract out the text, you can use the `.text` attribute.

In [ ]:
soup.find('title').text

The `.find` method find the first matching tag. 

We can find _all_ elements with a particular tag using the `.findAll(<tag>)` method. Say we want to find all images. We'll look for the `img` tag.

In [ ]:
images = soup.findAll('img')
images

Let's look closer at the first image.

In [ ]:
first_image = images[0]
first_image

You can access attributes of a Tag object in the same way that you would access values from a dictionary.

In [ ]:
first_image['src']

You can also safely access attributes using `.get`. This might be useful if, for example, you aren't sure if a particular Tag or all tags had a certain attribute.

In [ ]:
# Non-safe
first_image['class']

In [ ]:
# Safe
first_image.get('class')

You can also specify a default value when using `get`.

In [ ]:
first_image.get('class', default = 'No Class')

If you want to grab a particular attribute for all images, an easy way to do so is with a list comprehension.

In [ ]:
image_srcs = [x.get('src') for x in images]

In [ ]:
image_srcs

We can further navigate the html tree to extract out other bits of information.

When scraping from a web page, you should make use of "View Page Source" and/or "Inspect Element" in your web browswer.

For example, let's say we want to look at the second header on the page.

In [ ]:
soup.findAll('header')[1]

Similar to using `find` and `findall` in the full soup, we can use the `.find` method just within a Tag.

In [ ]:
soup.findAll('header')[1].find('h1').get('id')

In [ ]:
soup.findAll('header')[1].find('h1').text

Now, let's look for the table containing the Turing Award winners.

Using `.findAll` reveals that there are multiple tables on the page.

In [ ]:
soup.findAll('table')

If we know a bit more about what we are looking for, we can include an `attrs` argument and pass a dictionary. 

Go to the Turing award page in your browser, right click on the top of the table and choose "Inspect". You will notice that this table is defined with tag `<table class="wikitable">.` Armed with this information, we can narrow down our search.

In [ ]:
soup.find('table', attrs={'class' : 'wikitable'})

We can display the table by importing the `HTML` function.

In [ ]:
table_html = str(soup.find('table', attrs={'class' : 'wikitable'}))

from IPython.core.display import HTML

HTML(table_html)

However, this does not give us a way to work with the data in the table, only to display it.

If we want to interact with the table, we can use the _pandas_ `read_html` method.

In [ ]:
import pandas as pd
import io

In [ ]:
pd.read_html(io.StringIO(str(table_html)))[0]

You can even just point the read_html function to the URL containing the table you want.

In [ ]:
URL = 'http://en.wikipedia.org/wiki/Turing_Award'
pd.read_html(URL)[1]